In [9]:
import sys, pathlib
sys.path.insert(0, str(pathlib.Path.cwd().parent.parent))  # add repo root
from tsum import tsum
import torch
import json
import pandas as pd

from ndtools import fun_binary_graph as fbg # ndtools available at github.com/jieunbyun/network-datasets
from ndtools.graphs import build_graph
from pathlib import Path
import networkx as nx   

# Load data

In [10]:
DATASET = Path("data") 

nodes = json.loads((DATASET / "nodes.json").read_text(encoding="utf-8"))
edges = json.loads((DATASET / "edges.json").read_text(encoding="utf-8"))
probs_dict = json.loads((DATASET / "probs_bin.json").read_text(encoding="utf-8"))

# build base graph
G_base: nx.Graph = build_graph(nodes, edges, probs_dict)


In [11]:
row_names = list(edges.keys())
n_state = 2 # binary states

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
probs = [[probs_dict[n]['0']['p'], probs_dict[n]['1']['p']] for n in row_names]
probs = torch.tensor(probs, dtype=torch.float32, device=device)

In [12]:
#origin = 'n1'
origin = 'n52'
dests = ['n22', 'n66']
sys_surv_st = 1

def s_fun(comps_st):
    travel_time, sys_st, info = fbg.eval_travel_time_to_nearest(comps_st, G_base, origin, dests,
                                                         avg_speed=60, # km/h
                                                         target_max = [1.5, 0.5], # hours: it shouldn't take longer than this compared to the original travel time
                                                         length_attr = 'length_km')
    if sys_st >= sys_surv_st:
       path = info['path_filtered_edges'] 
       min_comps_st = {eid: ('>=', 1) for eid in path} # edges in the path are working
       min_comps_st['sys'] = ('>=', sys_st) # system edge is also working
    else:
        min_comps_st = None
    return travel_time, sys_st, min_comps_st

In [13]:
TSUMPATH = Path("tsum_res") 

rules_mat_surv = torch.load(TSUMPATH / f"rules_geq_{sys_surv_st}.pt", map_location="cpu")
rules_mat_surv = rules_mat_surv.to(device)
rules_mat_fail = torch.load(TSUMPATH / f"rules_leq_{sys_surv_st-1}.pt", map_location="cpu")
rules_mat_fail = rules_mat_fail.to(device)

# Calculate system probabilities

## Marginal probability

In [14]:
pr_cond = tsum.get_comp_cond_sys_prob(
    rules_mat_surv,
    rules_mat_fail,
    probs,
    comps_st_cond = {},
    row_names = row_names,
    s_fun = s_fun,
    sys_surv_st = sys_surv_st
)
print(f"P(sys >= {sys_surv_st}) = {pr_cond['survival']:.3e}")
print(f"P(sys <= {sys_surv_st-1} ) = {pr_cond['failure']:.3e}\n")

P(sys >= 1) = 9.987e-01
P(sys <= 0 ) = 1.284e-03



## Conditional probability given "one" component's survival

In [ ]:
for x in row_names:
    print(f"Eval P(sys | {x}=1)")
    pr_cond = tsum.get_comp_cond_sys_prob(
        rules_mat_surv,
        rules_mat_fail,
        probs,
        comps_st_cond = {x: 1},
        row_names = row_names,
        s_fun = s_fun,
        sys_surv_st = sys_surv_st
    )
    print(f"P(sys >= {sys_surv_st} | {x}=1) = {pr_cond['survival']:.3e}")
    print(f"P(sys <= {sys_surv_st-1} | {x}=1) = {pr_cond['failure']:.3e}\n")

Eval P(sys | e0001=1)
P(sys >= 1 | e0001=1) = 9.987e-01
P(sys <= 0 | e0001=1) = 1.261e-03

Eval P(sys | e0002=1)
P(sys >= 1 | e0002=1) = 9.987e-01
P(sys <= 0 | e0002=1) = 1.274e-03

Eval P(sys | e0003=1)
P(sys >= 1 | e0003=1) = 9.987e-01
P(sys <= 0 | e0003=1) = 1.303e-03

Eval P(sys | e0004=1)
P(sys >= 1 | e0004=1) = 9.987e-01
P(sys <= 0 | e0004=1) = 1.320e-03

Eval P(sys | e0005=1)
P(sys >= 1 | e0005=1) = 9.987e-01
P(sys <= 0 | e0005=1) = 1.279e-03

Eval P(sys | e0006=1)
P(sys >= 1 | e0006=1) = 9.987e-01
P(sys <= 0 | e0006=1) = 1.287e-03

Eval P(sys | e0007=1)
P(sys >= 1 | e0007=1) = 9.988e-01
P(sys <= 0 | e0007=1) = 1.204e-03

Eval P(sys | e0008=1)
P(sys >= 1 | e0008=1) = 9.988e-01
P(sys <= 0 | e0008=1) = 1.190e-03

Eval P(sys | e0009=1)
P(sys >= 1 | e0009=1) = 9.988e-01
P(sys <= 0 | e0009=1) = 1.208e-03

Eval P(sys | e0010=1)
P(sys >= 1 | e0010=1) = 9.988e-01
P(sys <= 0 | e0010=1) = 1.224e-03

Eval P(sys | e0011=1)
P(sys >= 1 | e0011=1) = 9.988e-01
P(sys <= 0 | e0011=1) = 1.212e-03


# Get multi-state probabilities

## First load rules for all states

In [16]:
TSUMPATH = Path("tsum_res") 
rules_dict_mat_surv = {}
rules_dict_mat_fail = {}

for sys_surv_st in [1, 2]:  # either 1 or 2
    rules_mat_surv = torch.load(TSUMPATH / f"rules_geq_{sys_surv_st}.pt", map_location="cpu")
    rules_mat_surv = rules_mat_surv.to(device)
    rules_dict_mat_surv[sys_surv_st] = rules_mat_surv
    rules_mat_fail = torch.load(TSUMPATH / f"rules_leq_{sys_surv_st-1}.pt", map_location="cpu")
    rules_mat_fail = rules_mat_fail.to(device)
    rules_dict_mat_fail[sys_surv_st] = rules_mat_fail

## Marginal probability

In [19]:

# Initialize empty list to store the results
results = []

# Calculate probabilities
cond_probs = tsum.get_comp_cond_sys_prob_multi(
                rules_dict_mat_surv,
                rules_dict_mat_fail,
                probs,
                comps_st_cond = {},
                row_names = row_names,
                s_fun = s_fun
            )

# Print results
print("P(sys):", cond_probs)

# Append data as a dictionary to the list
results.append({"System failure": cond_probs[0],
                "Partial failure": cond_probs[1],
                "Survival": cond_probs[2]
                })

# Convert the list to a DataFrame
df_results = pd.DataFrame(results)

# Save to a JSON file
df_results.to_json("post-processing/sys_probs.json", orient="records", indent=4)

print("\nData saved to post-processing/sys_probs.json")

P(sys): {0: 0.001258, 1: 0.000154, 2: 0.998588}

Data saved to post-processing/sys_probs.json


## Conditional probability given "one" component's survival
State 0: System failure\
State 1: Partial failure\
State 2: System survival

In [18]:
# Initialize empty list to store the results
results = []

for x in row_names:
    # Calculate probabilities
    cond_probs = tsum.get_comp_cond_sys_prob_multi(
                    rules_dict_mat_surv,
                    rules_dict_mat_fail,
                    probs,
                    comps_st_cond = {x: 0}, # 1: survival, 0: failure
                    row_names = row_names,
                    s_fun = s_fun
                )

    # Print results
    print(f"P(sys | {x}=0):", cond_probs)

    # Append data as a dictionary to the list
    results.append({"Component": x,
                    "System failure": cond_probs[0],
                    "Partial failure": cond_probs[1],
                    "Survival": cond_probs[2]
                    })

# Convert the list to a DataFrame
df_results = pd.DataFrame(results)

# Save to a JSON file
df_results.to_json("post-processing/cond_sys_probs.json", orient="records", indent=4)

print("\nData saved to post-processing/cond_sys_probs.json")

P(sys | e0001=0): {0: 0.001175, 1: 0.00012, 2: 0.998705}
P(sys | e0002=0): {0: 0.001312, 1: 0.000139, 2: 0.998549}
P(sys | e0003=0): {0: 0.001202, 1: 0.000137, 2: 0.998661}
P(sys | e0004=0): {0: 0.001191, 1: 0.000139, 2: 0.99867}
P(sys | e0005=0): {0: 0.001264, 1: 0.000149, 2: 0.998587}
P(sys | e0006=0): {0: 0.001223, 1: 0.000161, 2: 0.998616}
P(sys | e0007=0): {0: 0.001242, 1: 0.000161, 2: 0.998597}
P(sys | e0008=0): {0: 0.001218, 1: 0.000127, 2: 0.998655}
P(sys | e0009=0): {0: 0.001198, 1: 0.000152, 2: 0.99865}
P(sys | e0010=0): {0: 0.001244, 1: 0.000155, 2: 0.998601}
P(sys | e0011=0): {0: 0.001242, 1: 0.000144, 2: 0.998614}
P(sys | e0012=0): {0: 0.001309, 1: 0.000155, 2: 0.998536}
P(sys | e0013=0): {0: 0.001183, 1: 0.00013, 2: 0.998687}
P(sys | e0014=0): {0: 0.00122, 1: 0.000136, 2: 0.998644}
P(sys | e0015=0): {0: 0.001268, 1: 0.00016, 2: 0.998572}
P(sys | e0016=0): {0: 0.001269, 1: 0.000143, 2: 0.998588}
P(sys | e0017=0): {0: 0.001261, 1: 0.000157, 2: 0.998582}
P(sys | e0018=0): {0